<a href="https://colab.research.google.com/github/alexjihun8/template-exercise/blob/main/evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import importlib.util
import os
import time
import json
import ast


class Evaluator:
    def __init__(self, student_module_name):
        self.student_module_name = student_module_name
        self.student_module = self._import_student_module(student_module_name)
        self.allowed_imports = ['time', 'matplotlib.pyplot', 'sys', 'tqdm', 'random', 'json']
        self.disallowed_builtins = ['sorted']
        self.disallowed_methods = ['sort']
        self.problem_list = ['problem1a', 'problem1b', 'problem1c', 'problem2', 'problem3', 'problem5']
        self.input_data = { }
        '''problem1a': [[1, 5, 6, 3, 2, 2]]
            'problem1b': [[1, 3, 2, 5, 2, 6]],
            'problem1c': [[5, 2, 3, 2, 6, 1]]}'''
        self.expected_outputs = { }
        '''problem1a': [[1, 2, 2, 3, 5, 6]],
            'problem1b': [[1, 2, 2, 3, 5, 6]],
            'problem1c': [[1, 2, 2, 3, 5, 6]]}'''
        self.points = { }
        '''problem1a': [5],
            'problem1b': [10],
            'problem1c': [10]}'''
        self.time_limits = { }
        '''problem1a': [1],
            'problem1b': [2],
            'problem1c': [2]}'''

    def _import_student_module(self, module_name):
        module_path = f"{module_name}.py"
        if not os.path.exists(module_path):
            raise FileNotFoundError(f"Module {module_path} not found.")
        spec = importlib.util.spec_from_file_location(module_name, module_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        return module

    def _filter_imports_and_builtins(self, code):
        tree = ast.parse(code)
        for node in ast.walk(tree):
            if isinstance(node, ast.Import):
                for alias in node.names:
                    if alias.name not in self.allowed_imports:
                        raise ImportError(f"Unauthorized import: {alias.name}")
            elif isinstance(node, ast.ImportFrom):
                if node.module not in self.allowed_imports:
                    raise ImportError(f"Unauthorized import: {node.module}")
            elif isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
                if node.func.id in self.disallowed_builtins:
                    raise NameError(f"Unauthorized builtin function: {node.func.id}")
            elif isinstance(node, ast.Attribute):
                if node.attr in self.disallowed_methods:
                    raise NameError(f"Unauthorized method used: {node.attr}")


    def set_test_cases(self, test_cases_folder):
        for problem in self.problem_list:
            file_path = os.path.join(test_cases_folder, f"{problem}.json")
            if os.path.exists(file_path):
                with open(file_path) as file:
                    data = json.load(file)
                    self.input_data[problem] = data.get('input_data', [])
                    self.expected_outputs[problem] = data.get('expected_outputs', [])
                    self.points[problem] = data.get('points', [])
                    self.time_limits[problem] = data.get('time_limits', [])

    def _log(self, message, log_file, verbose=False):
        if verbose:
            print(message)
        with open(log_file, 'a') as file:
            file.write(message + "\n")

    def evaluate(self, target_problems="all", verbose=False):
        total_points = 0
        max_points = 0
        log_file = f"{self.student_module_name}_result.txt"
        open(log_file, 'w').close()  # Clear the log file

        for problem in self.problem_list:
            if target_problems != "all" and problem not in target_problems:
                continue

            if problem not in self.input_data:
                self._log(f"No test cases set for {problem}.", log_file, verbose=verbose)
                continue

            for idx, (test_case, expected_output, point, time_limit) in enumerate(zip(self.input_data[problem], self.expected_outputs[problem], self.points[problem], self.time_limits[problem])):
                problem_func = getattr(self.student_module, problem, None)
                if not problem_func:
                    self._log(f"No implementation for {problem}. Skipping...", log_file, verbose=verbose)
                    continue

                try:
                    start_time = time.time()
                    student_output = problem_func(test_case)
                    ### For Problem 3(e)
                    ### Comment the above line and Uncomment from here...
                    #l = [None, True]
                    #student_output = problem_func(*l)
                    ### Until here

                    elapsed_time = time.time() - start_time
                except Exception as e:
                    self._log(f"Error executing {problem} for test case {idx}: {e}", log_file, verbose=verbose)
                    continue

                if elapsed_time > time_limit:
                    self._log(f"Test case {idx} for {problem} exceeded the time limit of {time_limit} seconds. Time taken: {elapsed_time:.2f} seconds.", log_file, verbose=verbose)
                    continue

                if student_output == expected_output:
                    total_points += point
                else:
                    self._log(f"{problem} failed for test case {idx}. Expected {expected_output}, got {student_output}", log_file, verbose=verbose)

                max_points += point

        # Check for unauthorized imports and builtins
        with open(f"{self.student_module_name}.py") as file:
            code = file.read()
            try:
                self._filter_imports_and_builtins(code)
            except (ImportError, NameError) as e:
                self._log(str(e), log_file, verbose=verbose)
                total_points = 0

        self._log(f"Total points: {total_points}/{max_points}", log_file, verbose=verbose)
        return total_points

In [58]:
evaluator = Evaluator("hw2_2024_00000")
evaluator.set_test_cases("test_cases_folder")
evaluator.evaluate(target_problems="all", verbose=True)
# evaluator.evaluate(target_problems=["problem5"], verbose=False)

Total points: 23/23


23